<a href="https://colab.research.google.com/github/athabrani/DeepLearning/blob/main/02_Classification_CovertDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [2]:
url = "https://raw.githubusercontent.com/farrelrassya/teachingMLDL/main/02.%20Deep%20Learning/Dataset/compressed_data.csv.gz"
data = pd.read_csv(url, compression='gzip')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           581012 non-null  int64
 1   Aspect                              581012 non-null  int64
 2   Slope                               581012 non-null  int64
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64
 6   Hillshade_9am                       581012 non-null  int64
 7   Hillshade_Noon                      581012 non-null  int64
 8   Hillshade_3pm                       581012 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64
 10  Wilderness_Area1                    581012 non-null  int64
 11  Soil_Type1                          581012 non-null 

In [4]:
data.dropna()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,1,0,3
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,1,0,3
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,1,0,3
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,1,0,3


In [5]:
data.corr()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Cover_Type
Elevation,1.000000,0.015735,-0.242697,0.306229,0.093306,0.365559,0.112179,0.205887,0.059148,0.148022,...,0.083005,0.021107,0.035433,0.217179,0.193595,0.212612,0.238164,0.066550,-0.619374,-0.269554
Aspect,0.015735,1.000000,0.078728,0.017376,0.070305,0.025121,-0.579273,0.336103,0.646944,-0.109172,...,-0.021991,0.002281,-0.020398,0.017706,0.008294,-0.005866,0.055988,0.074904,0.082687,0.017080
Slope,-0.242697,0.078728,1.000000,-0.010607,0.274976,-0.215914,-0.327199,-0.526911,-0.175854,-0.185662,...,-0.022228,0.002918,0.007848,-0.072208,0.093602,0.025637,-0.036253,0.125663,0.255503,0.148285
Horizontal_Distance_To_Hydrology,0.306229,0.017376,-0.010607,1.000000,0.606236,0.072030,-0.027088,0.046790,0.052330,0.051874,...,-0.005231,0.033421,-0.006802,0.043031,0.031922,0.147020,0.055726,0.122028,-0.100433,-0.020317
Vertical_Distance_To_Hydrology,0.093306,0.070305,0.274976,0.606236,1.000000,-0.046372,-0.166333,-0.110957,0.034902,-0.069913,...,-0.006092,0.012955,-0.007520,-0.008629,0.043859,0.179006,-0.008709,0.146839,0.077792,0.081664
Horizontal_Distance_To_Roadways,0.365559,0.025121,-0.215914,0.072030,-0.046372,1.000000,0.034349,0.189461,0.106119,0.331580,...,-0.003000,0.007550,0.016313,0.079778,0.033762,0.016052,-0.200411,-0.232933,-0.270349,-0.153450
Hillshade_9am,0.112179,-0.579273,-0.327199,-0.027088,-0.166333,0.034349,1.000000,0.010037,-0.780296,0.132669,...,0.027870,0.007865,0.010332,0.015108,-0.029620,-0.000016,-0.006181,-0.100565,-0.200282,-0.035415
Hillshade_Noon,0.205887,0.336103,-0.526911,0.046790,-0.110957,0.189461,0.010037,1.000000,0.594274,0.057329,...,0.005863,0.016239,-0.022707,0.042952,-0.071961,-0.040176,0.042392,0.048646,-0.195733,-0.096426
Hillshade_3pm,0.059148,0.646944,-0.175854,0.052330,0.034902,0.106119,-0.780296,0.594274,1.000000,-0.047981,...,-0.016482,0.001330,-0.022064,0.022187,-0.029040,-0.024254,0.034707,0.090757,0.018860,-0.048290
Horizontal_Distance_To_Fire_Points,0.148022,-0.109172,-0.185662,0.051874,-0.069913,0.331580,0.132669,0.057329,-0.047981,1.000000,...,-0.000081,-0.010595,0.004180,-0.019740,-0.003301,0.008915,0.027473,-0.277510,-0.236548,-0.108936


In [6]:
X = data.drop('Cover_Type', axis=1)
y = data['Cover_Type'] - 1

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [8]:
Counter(y_train)

Counter({1: 226640, 6: 16408, 0: 169472, 4: 7594, 2: 28603, 3: 2198, 5: 13894})

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Klasifikasi PyTorch

In [11]:
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_train_torch = torch.tensor(y_train.values, dtype=torch.long)
y_test_torch = torch.tensor(y_test.values, dtype=torch.long)


In [12]:
class pyModel(nn.Module):
  def __init__(self):
    super(pyModel, self).__init__()
    self.fc1 = nn.Linear(X_train.shape[1], 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 64)
    self.out = nn.Linear(64, 7)
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.out(x)
    return x

torch_model = pyModel()

In [13]:
optimizer = optim.Adam(torch_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [14]:
train_dataset = TensorDataset(X_train_torch, y_train_torch)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

best_loss = float('inf')
patience = 5
counter = 0

for epoch in range(1, 200):
  torch_model.train()
  total_loss=0
  for x, target in train_loader:
    optimizer.zero_grad()
    output=torch_model(x)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()

  avg_loss = total_loss / len(train_loader)
  if epoch % 10 == 0:
    print(f'Epoch: {epoch}, Loss: {avg_loss:.4f}')

  if avg_loss < best_loss:
    best_loss = avg_loss
    best_model = torch_model.state_dict()
    counter = 0
  else:
    counter += 1
    if counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        torch_model.load_state_dict(best_model)
        break

Epoch: 10, Loss: 0.2267
Epoch: 20, Loss: 0.1792
Epoch: 30, Loss: 0.1564
Epoch: 40, Loss: 0.1427
Epoch: 50, Loss: 0.1332
Epoch: 60, Loss: 0.1265
Epoch: 70, Loss: 0.1205
Epoch: 80, Loss: 0.1165
Epoch: 90, Loss: 0.1121
Epoch: 100, Loss: 0.1090
Epoch: 110, Loss: 0.1060
Epoch: 120, Loss: 0.1036
Epoch: 130, Loss: 0.1013
Epoch: 140, Loss: 0.0998
Epoch: 150, Loss: 0.0970
Early stopping at epoch 155


In [16]:
y_pred_torch = torch.argmax(torch_model(X_test_torch), axis=1).numpy()
print("Akurasi:", accuracy_score(y_test, y_pred_torch))
print("Presisi:", precision_score(y_test, y_pred_torch, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_torch, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred_torch, average='weighted'))

Akurasi: 0.9478757002831252
Presisi: 0.9482553132902648
Recall: 0.9478757002831252
F1 Score: 0.9477839641806344


#Klasifikasi Model Tensorflow

In [17]:
X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test_tf = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.int32)
y_test_tf = tf.convert_to_tensor(y_test, dtype=tf.int32)

In [19]:
inputs = tf.keras.Input(shape=(X_train.shape[1], ))
hidden = tf.keras.layers.Dense(256, activation='relu')(inputs)
hidden = tf.keras.layers.Dense(128, activation='relu')(hidden)
hidden = tf.keras.layers.Dense(64, activation='relu')(hidden)
outputs = tf.keras.layers.Dense(7, activation='softmax')(hidden)

model = tf.keras.Model(inputs, outputs)


In [23]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    X_train,
    y_train,
    validation_split=0.15,
    batch_size=128,
    epochs=200,
    callbacks=[early_stopping]
)

Epoch 1/200
3087/3087 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.8634 - loss: 0.3291 - val_accuracy: 0.8739 - val_loss: 0.3099
Epoch 2/200
3087/3087 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.8788 - loss: 0.2957 - val_accuracy: 0.8777 - val_loss: 0.2994
Epoch 3/200
3087/3087 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.8877 - loss: 0.2735 - val_accuracy: 0.8888 - val_loss: 0.2728
Epoch 4/200
3087/3087 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.8959 - loss: 0.2557 - val_accuracy: 0.8944 - val_loss: 0.2598
Epoch 5/200
3087/3087 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.9017 - loss: 0.2414 - val_accuracy: 0.9028 - val_loss: 0.2394
Epoch 6/200
3087/3087 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.9053 - loss: 0.2321 - val_accuracy: 0.9032 - val_loss: 0.2423
Epoch 7/200
3087/3087 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.9084 - loss: 0.2245 - val_accuracy: 0.9062 - val_loss: 0.2355
Epoch 8/200
3087/3087 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.9130 -

In [25]:
y_pred_tf = np.argmax(model.predict(X_test_tf), axis=1)
print("Akurasi:", accuracy_score(y_test, y_pred_tf))
print("Presisi:", precision_score(y_test, y_pred_tf, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_tf, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred_tf, average='weighted'))


3632/3632 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step
Akurasi: 0.9368949166544753
Presisi: 0.9374074829884897
Recall: 0.9368949166544753
F1 Score: 0.9370505819083538


## **1. Pembuatan Model Deep Learning**
Dalam tugas ini, dua model deep learning dibuat menggunakan **PyTorch** dan **TensorFlow**.  
Keduanya digunakan untuk memprediksi variabel target `Cover Type` pada dataset dengan pendekatan **klasifikasi biner**.  
Model terdiri dari lapisan-lapisan **Dense** dengan fungsi aktivasi **ReLU**.
Model ini dilatih selama **200 epoch** dengan optimizer **Adam** dan fungsi loss **Crossentropy**.

---

## **2. Perbandingan Matriks Evaluasi**  
Berikut adalah hasil evaluasi kedua model:

| **Metric**   | **PyTorch** | **TensorFlow** |
|-------------|------------|---------------|
| **Akurasi**  | **0.9479**  | 0.9369        |
| **Presisi**  | **0.9483**  | 0.9374        |
| **Recall**   | **0.9479**  | 0.9369        |
| **F1 Score** | **0.9478**  | 0.9371        |

---

## **3. Analisis Perbandingan**
### **Akurasi (Accuracy)**
$$
Accuracy = \frac{TP + TN}{TP + TN + FP + FN}
$$
- **Akurasi mengukur seberapa sering model membuat prediksi yang benar.**
- **PyTorch memiliki akurasi lebih tinggi (0.9479) dibandingkan TensorFlow (0.9369), menunjukkan bahwa model PyTorch lebih baik dalam mengklasifikasikan data secara keseluruhan.**

### **Presisi (Precision)**
$$
Precision = \frac{TP}{TP + FP}
$$
- **Presisi menunjukkan seberapa banyak prediksi positif yang benar dibandingkan total prediksi positif.**
- **PyTorch memiliki presisi lebih tinggi (0.9483) dibandingkan TensorFlow (0.9374), yang berarti model PyTorch lebih sedikit menghasilkan false positive.**

### **Recall (Sensitivity)**
$$
Recall = \frac{TP}{TP + FN}
$$
- **Recall menunjukkan seberapa baik model dalam mendeteksi kelas positif.**
- **PyTorch memiliki recall lebih tinggi (0.9479) dibandingkan TensorFlow (0.9369), menunjukkan bahwa model PyTorch lebih baik dalam mendeteksi kelas positif dan mengurangi false negative.**

### **F1 Score**
$$
F1 = 2 \times \frac{Precision \times Recall}{Precision + Recall}
$$
- **F1 Score merupakan keseimbangan antara presisi dan recall.**
- **PyTorch memiliki F1 Score lebih tinggi (0.9478) dibandingkan TensorFlow (0.9371), yang menunjukkan bahwa model PyTorch lebih seimbang dalam memprediksi kelas positif dengan kesalahan yang lebih kecil.**

---

## **4. Kesimpulan Perbandingan Model**
Berdasarkan hasil evaluasi:
- **Model PyTorch memiliki akurasi, presisi, recall, dan F1 Score lebih tinggi dibandingkan model TensorFlow.**
- **Hasil ini menunjukkan bahwa model PyTorch lebih akurat, lebih baik dalam mendeteksi kelas positif, dan lebih seimbang dalam memprediksi kelas positif dan negatif dibandingkan TensorFlow.**
- **Secara keseluruhan, model PyTorch menunjukkan performa yang lebih unggul dibandingkan TensorFlow.**
